In [34]:
import numpy as np
import pandas as pd

from collections import defaultdict
from collections import Counter

import re

In [2]:
data_dir = "../data/Liu_2019"

hla_i_file = "optitype_combined_HLAs.csv"
ID_file = "SraRunTable_short.tsv"
table_s1 = "41591_2019_654_MOESM4_ESM.xlsx"

In [3]:
df_hla = pd.read_csv(data_dir+"/"+hla_i_file, header=0)
print(df_hla.shape)
df_hla[:6]

(288, 7)


,ID,A1,A2,B1,B2,C1,C2
0,SRR10801640,A*01:01,A*03:01,B*07:02,B*44:05,C*02:02,C*07:02
1,SRR10801641,A*01:01,A*02:01,B*44:02,B*52:01,C*05:01,C*12:02
2,SRR10801642,A*03:01,A*24:02,B*07:02,B*07:02,C*07:02,C*07:02
3,SRR10801643,A*24:02,A*31:01,B*38:01,B*44:02,C*12:03,C*05:01
4,SRR10801644,A*01:01,A*66:01,B*41:01,B*08:23,C*17:01,C*17:01
5,SRR10801645,A*02:01,A*03:02,B*44:02,B*51:01,C*15:02,C*05:01


In [4]:
df_ID = pd.read_csv(data_dir+"/"+"SraRunTable_short.tsv",sep='\t')
print(df_ID.shape)
df_ID[:6]

(288, 41)


,Run,analyte_type,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,biospecimen_repository,biospecimen_repository_sample_id,Bytes,...,submitted_subject_id,AssemblyName,study_disease,subject_is_affected,flowcell_barcode (run),instrument_name (run),lsid (exp),lsid (run),material_type (exp),biospecimen__repository_sample_type
0,SRR11080170,DNA; RNA,WXS,152,21110407976,PRJNA82747,SAMN12309122,NHGRI_Melanoma_Garraway,Patient142_T,6369252802,...,Patient142,GCA_000001405.13,Melanoma,Yes,"HK5C7BBXX\, HJYHVBBXX",SL-HFC,broadinstitute.org:bsp.prod.sample:EXZ57,broadinstitute.org:bsp.prod.sample:EXZ57,DNA:DNA Somatic,T
1,SRR11080191,DNA; RNA,WXS,152,24884983848,PRJNA82747,SAMN12309302,NHGRI_Melanoma_Garraway,Patient156_T,7505282208,...,Patient156,GCA_000001405.13,Melanoma,Yes,"HJYHVBBXX\, HK5C7BBXX",SL-HFC,broadinstitute.org:bsp.prod.sample:EXZ6D,broadinstitute.org:bsp.prod.sample:EXZ6D,DNA:DNA Somatic,T
2,SRR11080225,DNA; RNA,WXS,152,38197538744,PRJNA82747,SAMN12309266,NHGRI_Melanoma_Garraway,Patient21_T_M,7901881806,...,Patient21,GCA_000001405.13,Melanoma,Yes,"HHKHGBBXX\, HHKHKBBXX\, HHK23BBXX\, HHK22BBXX","SL-HFB\, SL-HFA",broadinstitute.org:bsp.prod.sample:DPR96,broadinstitute.org:bsp.prod.sample:DPR96,DNA:DNA Somatic,T_M
3,SRR11080226,DNA; RNA,WXS,152,26446046040,PRJNA82747,SAMN12309293,NHGRI_Melanoma_Garraway,Patient149_T,6976852028,...,Patient149,GCA_000001405.13,Melanoma,Yes,"HKKTJBBXX\, HKHVFBBXX\, HKKLMBBXX","SL-HFC\, SL-HFA",broadinstitute.org:bsp.prod.sample:F56LN,broadinstitute.org:bsp.prod.sample:F56LN,DNA:DNA Somatic,T
4,SRR10841991,DNA,WXS,152,32636008616,PRJNA82747,SAMN12309077,NHGRI_Melanoma_Garraway,Patient100_N,7086590983,...,Patient100,GCA_000001405.13,Melanoma,Yes,"HHKH7BBXX\, HHK22BBXX\, HHKHHBBXX","SL-HFA\, SL-HFB",broadinstitute.org:bsp.prod.sample:DPR7K,broadinstitute.org:bsp.prod.sample:DPR7K,DNA:DNA Somatic,N
5,SRR10841995,DNA,WXS,152,25095568448,PRJNA82747,SAMN12309100,NHGRI_Melanoma_Garraway,Patient112_N,5053428698,...,Patient112,GCA_000001405.13,Melanoma,Yes,"HHK23BBXX\, HHK22BBXX\, HHKHKBBXX\, HHKHGBBXX","SL-HFB\, SL-HFA",broadinstitute.org:bsp.prod.sample:DPSS4,broadinstitute.org:bsp.prod.sample:DPSS4,DNA:DNA Somatic,N


In [5]:
set(df_hla.ID) == set(df_ID.Run)

True

In [6]:
set(["_".join(x.split("_")[1:]) for x in df_ID.biospecimen_repository_sample_id.tolist()])

{'N', 'T', 'T_M', 'T_P'}

In [7]:
Counter(["_".join(x.split("_")[1:]) for x, y in zip(df_ID.biospecimen_repository_sample_id.tolist(), 
                                                    df_ID.Is_Tumor.tolist()) if y=="Yes"])

Counter({'T': 56, 'T_M': 49, 'T_P': 39})

In [8]:
Counter(["_".join(x.split("_")[1:]) for x, y in zip(df_ID.biospecimen_repository_sample_id.tolist(), 
                                                    df_ID.Is_Tumor.tolist()) if y=="No"])

Counter({'N': 144})

In [9]:
set(Counter(df_ID.submitted_subject_id.tolist()).values())

{2}

In [10]:
# build a dictionary with submitted subject id (patient***) as key and 
# biospecimen_repository_sample_id (patient***_ one of {'N', 'T', 'T_M', 'T_P'}) as key
# build another dictioary with biospecimen_repository_sample_id and 
# the corresponding run (SRR********) as value

bio_dict = defaultdict(list)

for b, c in zip(df_ID.biospecimen_repository_sample_id.tolist(), 
                df_ID.submitted_subject_id.tolist()):
    bio_dict[c] += [b]


run_dict = defaultdict(str)

for a, b in zip(df_ID.Run.tolist(), 
                df_ID.biospecimen_repository_sample_id.tolist()):
    run_dict[b] = a

In [11]:
bio_dict

defaultdict(list,
            {'Patient142': ['Patient142_T', 'Patient142_N'],
             'Patient156': ['Patient156_T', 'Patient156_N'],
             'Patient21': ['Patient21_T_M', 'Patient21_N'],
             'Patient149': ['Patient149_T', 'Patient149_N'],
             'Patient100': ['Patient100_N', 'Patient100_T_P'],
             'Patient112': ['Patient112_N', 'Patient112_T_P'],
             'Patient102': ['Patient102_N', 'Patient102_T_P'],
             'Patient108': ['Patient108_N', 'Patient108_T_P'],
             'Patient104': ['Patient104_N', 'Patient104_T_P'],
             'Patient105': ['Patient105_N', 'Patient105_T_P'],
             'Patient106': ['Patient106_N', 'Patient106_T_P'],
             'Patient10': ['Patient10_N', 'Patient10_T_M'],
             'Patient110': ['Patient110_N', 'Patient110_T_M'],
             'Patient107': ['Patient107_N', 'Patient107_T_P'],
             'Patient59': ['Patient59_N', 'Patient59_T_M'],
             'Patient36': ['Patient36_N', 'Patient36

In [12]:
# build a dict with run (SRR********) as key and the set of HLAs as value

hla_dict = defaultdict(list)

for cur_id, a1, a2, b1, b2, c1, c2 in zip(df_hla.ID.tolist(), 
                                          df_hla.A1.tolist(), 
                                          df_hla.A2.tolist(), 
                                          df_hla.B1.tolist(), 
                                          df_hla.B2.tolist(),
                                          df_hla.C1.tolist(), 
                                          df_hla.C2.tolist()):
    
    cur_hla_set = set([a1, a2, b1, b2, c1, c2])
    cur_hla_list = list(cur_hla_set)
    cur_hla_list.sort()
    hla_dict[cur_id] = cur_hla_list

In [13]:
# for each patient, check whether the HLAs are consistent between two samples

patient_ids = set(df_ID.submitted_subject_id)
patient_ids = list(patient_ids)
patient_ids.sort()

match_flags = []

for patient in patient_ids:
    
    bio_list = bio_dict[patient]
    bio_list.sort()
    
    run_list_0 = run_dict[bio_list[0]]
    run_list_1 = run_dict[bio_list[1]]
    
    hlas_0 = hla_dict[run_list_0]
    hlas_1 = hla_dict[run_list_1]
    
    if hlas_0 == hlas_1:
        match_flags += [True]
    else:
        match_flags += [False]
        print("              ")
        print(patient)
        print(bio_list[0]+": "+" ".join(hlas_0))
        print(bio_list[1]+": "+" ".join(hlas_1))

              
Patient117
Patient117_N: A*24:02 A*26:01 B*38:01 B*52:01 C*12:02 C*12:03
Patient117_T_P: A*24:02 A*26:01 B*38:01 B*52:01 C*12:02 C*12:07
              
Patient120
Patient120_N: A*24:02 A*30:04 B*35:12 B*58:01 C*03:02 C*04:01
Patient120_T_P: A*24:02 A*30:04 B*35:01 C*03:02 C*04:01
              
Patient170
Patient170_N: A*01:01 A*11:01 B*35:03 B*53:01 C*04:01
Patient170_T: A*01:01 A*11:01 B*35:03 B*35:29 C*04:01
              
Patient179
Patient179_N: A*01:01 A*02:01 B*08:01 B*15:01 C*03:04 C*07:01
Patient179_T: A*01:01 B*08:01 B*15:01 C*07:01
              
Patient75
Patient75_N: A*11:01 A*31:01 B*40:01 B*51:01 C*03:03 C*03:04
Patient75_T_P: A*11:01 A*31:01 B*40:01 B*51:01 C*03:04 C*03:43


In [14]:
Counter(match_flags)

Counter({True: 139, False: 5})

In [15]:
# build a dict with run (SRR********) as key and the string of sorted HLAs as value
# (convert to the format as in Chowell 2018 and 2019 tables)

def convert_hla(hla):
    return("".join([hla[0], hla[2:4], hla[5:]]))

hla_str_dict = defaultdict(str)

for cur_id, a1, a2, b1, b2, c1, c2 in zip(df_hla.ID.tolist(), 
                                          df_hla.A1.tolist(), 
                                          df_hla.A2.tolist(), 
                                          df_hla.B1.tolist(), 
                                          df_hla.B2.tolist(),
                                          df_hla.C1.tolist(), 
                                          df_hla.C2.tolist()):
    
    cur_hla_list = [convert_hla(a1), 
                    convert_hla(a2), 
                    convert_hla(b1), 
                    convert_hla(b2), 
                    convert_hla(c1), 
                    convert_hla(c2)]
    cur_hla_list.sort()
    hla_str_dict[cur_id] = ",".join(cur_hla_list)

In [16]:
hla_str_dict

defaultdict(str,
            {'SRR10801640': 'A0101,A0301,B0702,B4405,C0202,C0702',
             'SRR10801641': 'A0101,A0201,B4402,B5201,C0501,C1202',
             'SRR10801642': 'A0301,A2402,B0702,B0702,C0702,C0702',
             'SRR10801643': 'A2402,A3101,B3801,B4402,C0501,C1203',
             'SRR10801644': 'A0101,A6601,B0823,B4101,C1701,C1701',
             'SRR10801645': 'A0201,A0302,B4402,B5101,C0501,C1502',
             'SRR10801646': 'A0301,A0301,B0702,B3501,C0401,C0702',
             'SRR10801647': 'A1101,A3101,B4001,B5101,C0304,C0343',
             'SRR10801648': 'A0205,A2402,B3701,B4901,C0701,C1504',
             'SRR10801649': 'A1101,A3101,B4001,B5101,C0303,C0304',
             'SRR10801650': 'A0101,A2902,B4403,B5201,C0202,C1601',
             'SRR10801651': 'A0301,A3201,B0702,B5101,C0102,C0202',
             'SRR10801652': 'A1101,A3201,B1501,B4002,C0202,C0401',
             'SRR10801653': 'A0201,A2402,B1801,B4402,C0501,C0701',
             'SRR10801654': 'A2601,A3101,B070

In [17]:
# build a dictionary with patient ID as key, and the string of HLAs as key

# for the five patients with HLAs from tumor sample being inconsistent with
# those from normal sample, use those from tumor sample

final_hla_dict = defaultdict(str)


patient_ids = set(df_ID.submitted_subject_id)
patient_ids = list(patient_ids)
patient_ids.sort()

chosen_bio_list = []

for patient in patient_ids:
    
    bio_list = bio_dict[patient]
    bio_list.sort()
    
    chosen_bio_list += [bio_list[1]]
    
    run_list_1 = run_dict[bio_list[1]]
    
    hlas_1 = hla_str_dict[run_list_1]
    
    final_hla_dict[patient] = hlas_1

In [18]:
len(final_hla_dict)

144

In [19]:
final_hla_dict

defaultdict(str,
            {'Patient1': 'A0201,A1101,B1801,B4402,C0501,C1203',
             'Patient10': 'A0101,A0201,B0801,B1501,C0304,C0701',
             'Patient100': 'A0201,A0301,B0702,B2705,C0202,C0702',
             'Patient102': 'A0201,A2402,B0702,B1801,C0701,C0702',
             'Patient104': 'A0201,A3001,B1302,B4901,C0602,C0701',
             'Patient105': 'A0201,A0201,B0702,B1501,C0304,C0702',
             'Patient106': 'A0101,A3201,B0801,B3501,C0401,C0701',
             'Patient107': 'A0301,A3001,B1302,B3501,C0401,C0602',
             'Patient108': 'A0101,A0301,B0702,B0801,C0701,C0702',
             'Patient11': 'A0101,A0201,B3701,B4002,C0602,C0702',
             'Patient110': 'A0201,A2902,B4403,B5101,C1402,C1601',
             'Patient112': 'A1101,A3201,B0702,B3501,C0401,C0702',
             'Patient116': 'A0101,A0205,B0702,B4901,C0701,C0702',
             'Patient117': 'A2402,A2601,B3801,B5201,C1202,C1207',
             'Patient118': 'A0101,A1101,B3501,B5701,C0303,C0602

In [20]:
df_s1 = pd.read_excel(data_dir+"/"+table_s1, 
                      sheet_name='Supplemental Table 1', 
                      skiprows=2, 
                      skipfooter=14)  
print(df_s1.shape)
df_s1[:6]

(144, 46)


,Unnamed: 0,total_muts,nonsyn_muts,clonal_muts,subclonal_muts,heterogeneity,total_neoantigens,CNA_prop,"gender (Male=1, Female=0)",biopsy site,...,postCTLA4,postMAPKTx,postCombinedCTLA_PD1,numPriorTherapies,biopsy site_categ,biopsyContext (1=Pre-Ipi; 2=On-Ipi; 3=Pre-PD1; 4=On-PD1),daysBiopsyToPD1,daysBiopsyAfterIpiStart,purity,ploidy
0,Patient1,34,22,12,10,0.454545,49,0.321417,0,skin,...,0.0,0.0,0,1,skin,3,-84,unk,0.92,1.73
1,Patient10,96,71,48,22,0.314286,230,0.391384,0,skin,...,1.0,1.0,0,2,skin,3,-12,107,0.83,1.84
2,Patient100,200,126,98,24,0.196721,301,0.029447,0,skin,...,0.0,0.0,0,1,skin,3,-94,33,0.11,2.17
3,Patient102,370,246,215,26,0.107884,825,0.169389,1,brain,...,0.0,0.0,0,0,brain,3,-64,na,0.70,3.24
4,Patient104,130,96,65,28,0.301075,329,0.206518,0,lymph node,...,0.0,0.0,0,0,lymph node,3,-57,na,0.86,4.58
5,Patient105,185,125,85,23,0.212963,334,0.394306,0,skin,...,0.0,0.0,1,0,skin,3,-22,na,0.86,2.42


In [21]:
set(df_s1['Unnamed: 0']) == set(df_ID['submitted_subject_id'])

True

In [22]:
set(df_s1['Unnamed: 0']) == set(final_hla_dict.keys())

True

In [23]:
hla_column = [final_hla_dict[x] for x in df_s1['Unnamed: 0'].tolist()]

In [24]:
df_s1["hla_class_i_alleles"] = hla_column
df_s1

,Unnamed: 0,total_muts,nonsyn_muts,clonal_muts,subclonal_muts,heterogeneity,total_neoantigens,CNA_prop,"gender (Male=1, Female=0)",biopsy site,...,postMAPKTx,postCombinedCTLA_PD1,numPriorTherapies,biopsy site_categ,biopsyContext (1=Pre-Ipi; 2=On-Ipi; 3=Pre-PD1; 4=On-PD1),daysBiopsyToPD1,daysBiopsyAfterIpiStart,purity,ploidy,hla_class_i_alleles
0,Patient1,34,22,12,10,0.454545,49,0.321417,0,skin,...,0.0,0,1,skin,3,-84,unk,0.92,1.73,"A0201,A1101,B1801,B4402,C0501,C1203"
1,Patient10,96,71,48,22,0.314286,230,0.391384,0,skin,...,1.0,0,2,skin,3,-12,107,0.83,1.84,"A0101,A0201,B0801,B1501,C0304,C0701"
2,Patient100,200,126,98,24,0.196721,301,0.029447,0,skin,...,0.0,0,1,skin,3,-94,33,0.11,2.17,"A0201,A0301,B0702,B2705,C0202,C0702"
3,Patient102,370,246,215,26,0.107884,825,0.169389,1,brain,...,0.0,0,0,brain,3,-64,na,0.70,3.24,"A0201,A2402,B0702,B1801,C0701,C0702"
4,Patient104,130,96,65,28,0.301075,329,0.206518,0,lymph node,...,0.0,0,0,lymph node,3,-57,na,0.86,4.58,"A0201,A3001,B1302,B4901,C0602,C0701"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Patient9,911,652,562,80,0.124611,1996,0.111034,0,skin,...,0.0,0,0,skin,3,-39,na,0.88,2.02,"A0201,A2402,B1801,B4402,C0501,C0701"
140,Patient94,414,268,227,21,0.084677,929,0.902095,0,skin,...,0.0,0,2,skin,3,-31,140,0.85,1.13,"A0301,A1101,B0702,B1501,C0303,C0702"
141,Patient96,959,649,588,50,0.078370,2136,0.196499,1,pancreas,...,0.0,0,0,liver/visceral,3,-81,na,0.82,2.22,"A0101,A1101,B3901,B5701,C0602,C0702"
142,Patient98,472,302,219,74,0.252560,790,0.223074,0,lung,...,0.0,1,0,lung,3,-24,na,0.60,1.99,"A2601,A3101,B0702,B5101,C0702,C1502"


In [25]:
df_s1.isna().sum()

Unnamed: 0                                                  0
total_muts                                                  0
nonsyn_muts                                                 0
clonal_muts                                                 0
subclonal_muts                                              0
heterogeneity                                               0
total_neoantigens                                           0
CNA_prop                                                    0
gender (Male=1, Female=0)                                   0
biopsy site                                                 0
monthsBiopsyPreTx                                           0
BR                                                          0
PFS                                                         0
OS                                                          0
TimeToBR                                                    0
cyclesOnTherapy                                             0
txOngoin

In [28]:
homozygous_flags = []

for hla_string in df_s1["hla_class_i_alleles"].tolist():
    hlas = hla_string.split(",")
    if ((hlas[0]!= hlas[1]) and (hlas[2]!=hlas[3]) and (hlas[4]!=hlas[5])):
        homozygous_flags += [0]
    else:
        homozygous_flags += [1]

In [29]:
Counter(homozygous_flags)

Counter({0: 107, 1: 37})

In [32]:
37/144

0.2569444444444444

In [31]:
df_s1["homozygous"] = homozygous_flags

In [35]:
# only keep the subjects with hla i alleles among the 85 ones with pseudo sequences
# or the hlas that we can find a replacement for from the hla alleles that we have pseudo sequence for

df_pseudo = pd.read_csv("../data/for_encoders/HLA_I_pseudo_40.csv", header = 0)
hla_i_considered = set(["".join(re.split('-|\*|:', x)[1:]) for x in df_pseudo.hla.tolist()])

In [36]:
liu_2019_hla_group_list = [x.split(",") for x in df_s1["hla_class_i_alleles"].tolist()]
liu_2019_hla_list = [y for x in liu_2019_hla_group_list for y in x]

print(len(liu_2019_hla_list))
print(len(set(liu_2019_hla_list)))

864
86


In [37]:
# there are 18 alleles from liu 2019 that are not included in the 85 ones
len(set(liu_2019_hla_list) - hla_i_considered)

18

In [38]:
len(hla_i_considered - set(liu_2019_hla_list))

17

In [39]:
temp_hla_candidate_list = list(set(liu_2019_hla_list) - hla_i_considered)
temp_hla_candidate_list.sort()
temp_hla_candidate_list

['A0217',
 'A3004',
 'A3402',
 'B0709',
 'B0823',
 'B1803',
 'B2702',
 'B3529',
 'B4405',
 'B4429',
 'B5107',
 'B5108',
 'B7301',
 'C0308',
 'C0343',
 'C1202',
 'C1207',
 'C1504']

In [40]:
hla_i_considered_list = list(hla_i_considered)
hla_i_considered_list.sort()
hla_i_considered_list

['A0101',
 'A0201',
 'A0205',
 'A0206',
 'A0301',
 'A0302',
 'A1101',
 'A2301',
 'A2402',
 'A2403',
 'A2501',
 'A2601',
 'A2901',
 'A2902',
 'A3001',
 'A3002',
 'A3101',
 'A3201',
 'A3301',
 'A3303',
 'A3401',
 'A6601',
 'A6801',
 'A6802',
 'B0702',
 'B0705',
 'B0801',
 'B1302',
 'B1401',
 'B1402',
 'B1501',
 'B1503',
 'B1507',
 'B1517',
 'B1518',
 'B1801',
 'B2705',
 'B3501',
 'B3502',
 'B3503',
 'B3508',
 'B3701',
 'B3801',
 'B3802',
 'B3901',
 'B3906',
 'B4001',
 'B4002',
 'B4006',
 'B4101',
 'B4102',
 'B4402',
 'B4403',
 'B4501',
 'B4801',
 'B4901',
 'B5001',
 'B5101',
 'B5201',
 'B5301',
 'B5501',
 'B5601',
 'B5701',
 'B5801',
 'C0102',
 'C0202',
 'C0302',
 'C0303',
 'C0304',
 'C0401',
 'C0501',
 'C0602',
 'C0701',
 'C0702',
 'C0704',
 'C0801',
 'C0802',
 'C0803',
 'C1203',
 'C1402',
 'C1502',
 'C1505',
 'C1601',
 'C1602',
 'C1701']

In [41]:
hla_replace_dict = defaultdict(str)

hla_replace_dict['A0217'] = 'A0201'
hla_replace_dict['A3004'] = 'A3001'
hla_replace_dict['A3402'] = 'A3401'


hla_replace_dict['B0709'] = 'B0702'
hla_replace_dict['B0823'] = 'B0801'

hla_replace_dict['B1803'] = 'B1801'

hla_replace_dict['B2702'] = 'B2705'

hla_replace_dict['B3529'] = 'B3501'

hla_replace_dict['B4405'] = 'B4402'
hla_replace_dict['B4429'] = 'B4402'

hla_replace_dict['B5107'] = 'B5101'
hla_replace_dict['B5108'] = 'B5101'


hla_replace_dict['C0308'] = 'C0302'
hla_replace_dict['C0343'] = 'C0302'

hla_replace_dict['C1202'] = 'C1203'
hla_replace_dict['C1207'] = 'C1203'
hla_replace_dict['C1504'] = 'C1502'

# 'B7301' does not have a match from the 85 HLAs considered

In [42]:
# use a function to find matching hla allele with pseudo sequence, 
# and build the dictionary again, to verify whether the previously built 
# dictionary is correct

# map each two digit HLA allele to its first match among the alleles

two_digit_dict = defaultdict(str)

all_two_digits = list(set([x[:3] for x in hla_i_considered_list]))

for a01 in all_two_digits:
    
    cur_list = [x for x in hla_i_considered_list if x[:3]==a01]
    
    cur_list.sort()
    
    two_digit_dict[a01] = cur_list[0]

In [43]:
print(len(all_two_digits))
print(len(two_digit_dict))

54
54


In [44]:
hla_replace_dict_verify = defaultdict(str)

for temp_hla in temp_hla_candidate_list:
    
    if temp_hla[:3] in set(all_two_digits):
        
        hla_replace_dict_verify[temp_hla] = two_digit_dict[temp_hla[:3]]

len(hla_replace_dict_verify)

17

In [45]:
temp_keys = list(hla_replace_dict.keys())

In [46]:
# now we have verified that the translation dictionary is correct
print(set(hla_replace_dict.keys()) == set(hla_replace_dict_verify.keys()))
sum([hla_replace_dict[x]==hla_replace_dict_verify[x] for x in temp_keys])

True


17

In [47]:
# output the list of hla alleles that either have pseudo sequences or can find a match

hla_alleles_allowed = hla_i_considered_list + temp_keys
hla_alleles_allowed.sort()
print(len(hla_alleles_allowed))
print(len(set(hla_alleles_allowed)))

102
102


In [50]:
# we find match for all hla alleles from liu 2019, except 'B7301'

# count how many subjects from liu 2019 have hla alleles all under our consideration
# or belong to one of those that we can find match for

hla_alleles_allowed_set = set(hla_alleles_allowed)

subjects_to_consider = []

for name, hlas in zip(df_s1['Unnamed: 0'].tolist(), df_s1["hla_class_i_alleles"].tolist()):
    cur_hla_set = set(hlas.split(","))
    if (len(cur_hla_set.intersection(hla_alleles_allowed_set)) == len(cur_hla_set)):
        subjects_to_consider += [name]

In [51]:
len(subjects_to_consider)

143

In [52]:
set(df_s1['Unnamed: 0'].tolist()) - set(subjects_to_consider)

{'Patient187'}

In [53]:
final_hla_dict['Patient187']

'A0201,A2402,B1803,B7301,C0701,C1505'

In [54]:
# subset df_2019 to only keep those with HLAs that we have pseudo amino acid information for
# or can find a match with pseudo information for
df_liu_2019_kept = df_s1.loc[df_s1['Unnamed: 0'].isin(subjects_to_consider)]
df_liu_2019_kept.shape

(143, 48)

In [57]:
Counter([a==b for a, b in zip(df_liu_2019_kept['Unnamed: 0'].tolist(), 
                              subjects_to_consider)])

Counter({True: 143})

In [58]:
df_liu_2019_kept[:2]

,Unnamed: 0,total_muts,nonsyn_muts,clonal_muts,subclonal_muts,heterogeneity,total_neoantigens,CNA_prop,"gender (Male=1, Female=0)",biopsy site,...,postCombinedCTLA_PD1,numPriorTherapies,biopsy site_categ,biopsyContext (1=Pre-Ipi; 2=On-Ipi; 3=Pre-PD1; 4=On-PD1),daysBiopsyToPD1,daysBiopsyAfterIpiStart,purity,ploidy,hla_class_i_alleles,homozygous
0,Patient1,34,22,12,10,0.454545,49,0.321417,0,skin,...,0,1,skin,3,-84,unk,0.92,1.73,"A0201,A1101,B1801,B4402,C0501,C1203",0
1,Patient10,96,71,48,22,0.314286,230,0.391384,0,skin,...,0,2,skin,3,-12,107,0.83,1.84,"A0101,A0201,B0801,B1501,C0304,C0701",0


In [59]:
df_liu_2019_kept.isna().sum()

Unnamed: 0                                                  0
total_muts                                                  0
nonsyn_muts                                                 0
clonal_muts                                                 0
subclonal_muts                                              0
heterogeneity                                               0
total_neoantigens                                           0
CNA_prop                                                    0
gender (Male=1, Female=0)                                   0
biopsy site                                                 0
monthsBiopsyPreTx                                           0
BR                                                          0
PFS                                                         0
OS                                                          0
TimeToBR                                                    0
cyclesOnTherapy                                             0
txOngoin

In [60]:
# translate hla alleles to those we have pseudo sequence for

kept_trans_alleles = []

for x in df_liu_2019_kept["hla_class_i_alleles"].tolist():
    
    cur_alleles = x.split(",")
    cur_trans_alleles = [hla if hla in hla_i_considered else hla_replace_dict[hla] for hla in cur_alleles]
    kept_trans_alleles += [cur_trans_alleles]

In [61]:
len(set([y for x in kept_trans_alleles for y in x]))

68

In [63]:
len(set(hla_i_considered) - set([y for x in kept_trans_alleles for y in x]))

17

In [64]:
kept_trans_alleles_string = [",".join(x) for x in kept_trans_alleles]
kept_trans_alleles_string[3:6]

['A0201,A2402,B0702,B1801,C0701,C0702',
 'A0201,A3001,B1302,B4901,C0602,C0701',
 'A0201,A0201,B0702,B1501,C0304,C0702']

In [65]:
df_liu_2019_kept["hla_class_i_alleles"].tolist()[3:6]

['A0201,A2402,B0702,B1801,C0701,C0702',
 'A0201,A3001,B1302,B4901,C0602,C0701',
 'A0201,A0201,B0702,B1501,C0304,C0702']

In [66]:
df_liu_2019_kept['hla_class_i_alleles_replace'] = kept_trans_alleles_string

/var/folders/2w/md4qmqxd3cn9kfl7flgbmyrw0000gp/T/ipykernel_76219/159923471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liu_2019_kept['hla_class_i_alleles_replace'] = kept_trans_alleles_string


In [67]:
list(zip(df_liu_2019_kept['hla_class_i_alleles'], 
         df_liu_2019_kept['hla_class_i_alleles_replace']))[:3]

[('A0201,A1101,B1801,B4402,C0501,C1203',
  'A0201,A1101,B1801,B4402,C0501,C1203'),
 ('A0101,A0201,B0801,B1501,C0304,C0701',
  'A0101,A0201,B0801,B1501,C0304,C0701'),
 ('A0201,A0301,B0702,B2705,C0202,C0702',
  'A0201,A0301,B0702,B2705,C0202,C0702')]

In [68]:
Counter([x==y for x,y in zip(df_liu_2019_kept['hla_class_i_alleles_replace'].tolist(), 
                             kept_trans_alleles_string)])

Counter({True: 143})

In [69]:
df_liu_2019_kept.to_csv("../results/st15_liu_2019_kept_hla_match_replace.csv", index = False)